<a href="https://colab.research.google.com/github/DavydovichYana/ColingML/blob/main/Spell_checking_Yana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# В этом задании вам необходимо будет реализовать статистический Spell Checker

In [2]:
!pip install razdel

import razdel
import numpy as np
import nltk
import tqdm
import scipy, re
import pandas as pd
from collections import Counter
from scipy.spatial.distance import euclidean
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
# Данные: Корпус русских текстов для n-gram статистики --> возьмем новостный корпус с corus
# Словарь слов русского языка (чем больше, тем лучше)
# Предложения, которые необходимо исправить

In [21]:
# Шаг 1:

# На первом шаге коррекции наших текстов определим такие токены, которым требуется исправление.
# Для этого проведем статистическую бинарную классификацию токенов в наших предложениях
# (1- токен содержит опечатку, 0- токен не содержит опечатку)

# Определять неправильные токены будем с помощью формулы расчета "подозрительности" триграмм из статьи 1975 года 
# "Computer Detection of Typographical Errors  R. Morris, L. Cherry". Статья приложена.

# Сначала напишем формулу для получения n-gram слова. Для формулы нам нужны только биграммы и триграммы, но мы напишем
# функцию, которая возвращает n-граммы для любого заданного n.

In [3]:
def ngram(word, n):
    ngrams = []
    if len(word) == n:
      ngrams.append(word)
    elif len(word) > n:
      for i in range(len(word)-n+1):
        ngrams.append(word[i:i+n])
    return ngrams


# как --> ['как']
# не --> []
# шарик --> ['шар', 'ари', 'рик']
# неправильный --> ['неп', 'епр', 'пра', 'рав', 'ави', 'вил', 'иль', 'льн', 'ьны', 'ный']


assert ngram('неправильный', 3) == [''.join(g) for g in list(nltk.ngrams('неправильный', 3))]

In [4]:
ngram('Абрикос', 3)

['Абр', 'бри', 'рик', 'ико', 'кос']

In [24]:
# Логика сбора статистики такова:
# 1) Идем по текстам корпуса новостей
# 2) Токенизируем тексты с помощью razdel.tokenize()
# 3) Приводим каждый токен к нижнему регистру
# 4) Токены, которые содержат только символы кириллицы, копим в статистику 
#    (делим токен на биграмы и триграммы и копим статистику в Counter)

In [5]:
# Корпус русских текстов (загружаю вручную, т.к. не получилось через corus)
records = pd.read_csv('lenta-ru-news.csv.gz', compression='gzip')
records.head()

,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/14/cancer/,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия,Общество
1,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды
2,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир
3,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика
4,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество


In [6]:
records.text.fillna('q', inplace = True)
records.text.isna().sum()

0

In [7]:
def clean(text):
  text = str(text).lower()
  text = re.sub("&lt;/?&gt;", " &lt;&gt ", text) #escaping html tags
  text = re.sub("(\\d|\\W)+", " ", text)
  text = razdel.tokenize(text)
  text = [i.text for i in text]
  return text

def is_cyrillic(word):
    return bool(re.fullmatch('^[а-яА-Я]+$', word))


#Проверка
cl = clean('CNN рассказал!!! о ))))намерении США отключить Россию от SWIFT в качестве санкции')
for i in cl:
  print(is_cyrillic(i))

False
True
True
True
True
True
True
True
False
True
True
True


In [8]:
ngram_stats = Counter()

for text in tqdm.tqdm(records.text):
    #TODO: на основе текстов создать статистику триграм 
    # Токенизируем текст
    tokens = clean(text)
    
    for token in tokens:
        if not is_cyrillic(token):
            continue

        # Получаем триграммы и биграммы
        for i in [2, 3]:
            n_grams = ngram(token,i)
            for g in n_grams:
                ngram_stats[g] += 1
        
        
print(ngram_stats)
                  

100%|██████████| 739351/739351 [36:56<00:00, 333.61it/s]

Counter({'ст': 12254576, 'ен': 9924095, 'ни': 9607686, 'ов': 9332128, 'ро': 9166699, 'ра': 9122033, 'на': 9072112, 'то': 9037227, 'ко': 8465830, 'но': 8341376, 'по': 8339967, 'ан': 7857828, 'ре': 7789186, 'пр': 7769505, 'ос': 6903801, 'ер': 6749466, 'та': 6629848, 'го': 6599923, 'ли': 6306905, 'ет': 5935478, 'ор': 5924792, 'ал': 5779030, 'од': 5511170, 'ом': 5472296, 'не': 5353062, 'ва': 5307744, 'те': 5262708, 'ле': 5176594, 'во': 5121104, 'ол': 5019903, 'де': 4856464, 'ка': 4828969, 'ск': 4788167, 'от': 4777668, 'ин': 4703421, 'ел': 4681263, 'ны': 4675590, 'ти': 4621878, 'ри': 4586772, 'ла': 4567366, 'он': 4491692, 'ль': 4488785, 'ит': 4441551, 'об': 4434764, 'за': 4375287, 'ил': 4290543, 'ат': 4290480, 'да': 4146275, 'ес': 4096290, 'ве': 4044313, 'ме': 4043771, 'со': 3998970, 'ог': 3992970, 'ло': 3892285, 'ед': 3820118, 'тр': 3750557, 'ар': 3581227, 'ав': 3489969, 'ак': 3443897, 'ми': 3417520, 'ени': 3387654, 'до': 3365209, 'ой': 3298535, 'ки': 3287459, 'ис': 3251322, 'ас': 3228504,

In [9]:
df = pd.read_csv('broken_texts.csv.gz', compression='gzip')[['text']]
df['text'].iloc[11]

'в 1996 г . плоучил звание заслуженныйй профессор харьковского государственного университета .'

Формула для расчета подозрительности триграмы (обозначается xyz) выглядит следующим образом:

$$ i(T) = \frac{1}{2}[log(xy) + log(yz)] - log(xyz) $$

Если биграма или триграма отсутствует в словаре, то значение логарифма по задумке авторов сразу равно -10

Эту логику лучше вынести в отдельную функцию

In [10]:
def count_log(xyz):
    
    if xyz in ngram_stats:
        xy,yz = ngram(xyz,2)
        log_xy,log_yz,log_xyz = list(map(np.log, [ngram_stats[xy],ngram_stats[yz],ngram_stats[xyz]]))
    
    else:
        log_xyz = -10
        xy,yz = ngram(xyz,2)
        log_xy = np.log(ngram_stats[xy]) if xy in ngram_stats else -10
        log_yz = np.log(ngram_stats[yz]) if yz in ngram_stats else -10
        
    return log_xy,log_yz,log_xyz

In [11]:
# Соберем всё вместе

def pecularity(trigram):
    
    log_xy,log_yz,log_xyz = count_log(trigram)
    return (0.5*(log_xy+log_yz)-log_xyz)


def get_scores(token):
    
    trigrams_list = ngram(token,3)
    scores_trigrams, token_dict = dict(),dict()
    
    for trigram in trigrams_list:
        scores_trigrams[trigram] = pecularity(trigram)
        
    token_dict[token] = scores_trigrams
        
    return token_dict


        
#    В конечном итоге скоры для одного слова должны выглядеть как-то так:
#     {'плоучил': 
#          {'пло': 2.59,
#           'лоу': 3.29,
#           'оуч': 4.09,
#           'учи': 1.56,
#           'чил': 2.40}
#     }
# Если токен имеет триграмы с скорами > 4, то мы считаем, что такой токен имеет ошибку.
# То есть его частота в нашем корпусе частот практически незначительна

In [12]:
get_scores('васпользаваться')

{'васпользаваться': {'ава': 2.5803540257894646,
  'асп': 2.5455738454594066,
  'ать': 1.2081999135495618,
  'вас': 4.838926246742796,
  'ват': 2.0171839334278054,
  'зав': 2.926890468611578,
  'льз': 1.5329396339825259,
  'оль': 1.493132831479965,
  'пол': 1.452875499126133,
  'спо': 1.8230202889697544,
  'тьс': 1.5776915607652278,
  'ьза': 6.1138220717879115,
  'ься': 1.5129040094333757}}

In [ ]:
# Токены, в которых есть значения выше 4: пробуем восстановить

# По аналогии с решением предыдущей задачи воспользуйтесь n-gram преобразованием, чтобы найти top-k ближайших кандидатов
# для исправления токена с помощью списка слов русского языка и функции scipy.cdist 

# Будем использовать и униграмы, и биграмы, и триграмы для этой части задания




In [13]:
# Слова русского языка
words = list(pd.read_csv('russian_words.zip', compression='zip').values.flatten())

print(words[120:130])

len(words)

['Абакан', 'Абакана', 'Абакане', 'абаканец', 'Абаканом', 'абаканская', 'абаканские', 'абаканский', 'абаканским', 'абаканскими']


1532628

In [14]:
words = clean(words)

In [22]:
vectorizer = CountVectorizer(analyzer='char',ngram_range=(1,3))
words_matrix = vectorizer.fit_transform(words)

In [23]:
vectorizer.vocabulary_

{'к': 4583,
 'а': 0,
 'ка': 4584,
 'л': 4953,
 'и': 3696,
 'б': 697,
 'о': 6456,
 'ли': 5093,
 'иб': 3718,
 'бо': 871,
 'либ': 5095,
 'ибо': 3729,
 'н': 5921,
 'у': 9167,
 'д': 1839,
 'ь': 11606,
 'ни': 6065,
 'бу': 951,
 'уд': 9255,
 'дь': 2243,
 'ниб': 6067,
 'ибу': 3732,
 'буд': 956,
 'удь': 9280,
 'с': 8164,
 'т': 8649,
 'та': 8650,
 'ак': 256,
 'ки': 4684,
 'так': 8660,
 'аки': 263,
 'то': 8850,
 'ко': 4757,
 'он': 6797,
 'нт': 6221,
 'кон': 4771,
 'онт': 6815,
 'нто': 6232,
 'я': 12497,
 'ля': 5432,
 'р': 7528,
 'аа': 1,
 'ар': 417,
 'ро': 7808,
 'на': 5922,
 'аар': 11,
 'аро': 431,
 'рон': 7822,
 'она': 6798,
 'в': 1096,
 'но': 6146,
 'ов': 6510,
 'оно': 6811,
 'нов': 6149,
 'аб': 17,
 'ба': 698,
 'аба': 18,
 'ж': 3015,
 'аж': 155,
 'жу': 3232,
 'ур': 9512,
 'баж': 705,
 'ажу': 169,
 'жур': 3247,
 'ра': 7529,
 'ура': 9513,
 'м': 5482,
 'ам': 311,
 'рам': 7542,
 'ми': 5599,
 'ами': 319,
 'х': 10008,
 'ах': 554,
 'рах': 7551,
 'е': 2316,
 'ре': 7636,
 'уре': 9518,
 'рн': 7788,
 'а

In [17]:
type(words_matrix)

scipy.sparse.csr.csr_matrix

In [ ]:
vectorizer.get_feature_names()

In [24]:
y1 = vectorizer.transform(['яблако']).toarray()
y2 = vectorizer.transform(['яблоко']).toarray()
scipy.spatial.distance.cdist(y1,y2)

array([[3.46410162]])

In [26]:
def predict_candidates(word, k):
    # TODO: predict top k most similar words from russian word dictionary
    cands = {}
    word_vector = vectorizer.transform([word]).toarray()
    for i, line in tqdm.tqdm(enumerate(words_matrix)):
      cands[words[i]] = scipy.spatial.distance.cdist(word_vector, line.toarray())

    candidates = sorted(cands.items(), key=lambda x: x[1])[:k]
    return candidates

In [27]:
predict_candidates('алимпиада', 10)

1536029it [03:16, 7835.12it/s]


[('олимпиада', array([[2.44948974]])),
 ('олимпиадам', array([[3.]])),
 ('олимпиадах', array([[3.]])),
 ('олимпиад', array([[3.31662479]])),
 ('олимпиадами', array([[3.46410162]])),
 ('олимпиаде', array([[3.74165739]])),
 ('олимпиаду', array([[3.74165739]])),
 ('олимпиады', array([[3.74165739]])),
 ('илиада', array([[3.87298335]])),
 ('падали', array([[3.87298335]]))]